In [ ]:
 #!pip install simple_salesforce

In [ ]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
 # Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [ ]:
 from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [ ]:
 engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [ ]:
 course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

In [ ]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

In [ ]:
 course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

In [ ]:
 course_data_records = course_data_df.to_dict('records')
course_data_records

In [ ]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)

In [ ]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)

In [ ]:
 course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

In [ ]:
query = '''
    SELECT
        c.*
        ,co.CourseCode
    FROM
        class c
        INNER JOIN course co
        ON c.ID_Course = co.ID_Course
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

In [ ]:
 class_data_df.rename(columns={
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)

class_data_df

In [ ]:
 class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

In [ ]:
 class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()

In [ ]:
 class_data_records = class_data_df.to_dict(orient='records')
class_data_records

In [ ]:
for rec in class_data_records:
 
    record = {
        'Course__c': rec['Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)